# Example of BOMD MD (NVE, NVT)

In [1]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")

import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)

device = torch.device('cpu')
# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
#     device = torch.device('cpu')

In [2]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                           # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                           # [1], adaptive mixing
                                           # [1, K, L, M] # advanced adaptive mixing.
                                           # First, it does linear mixing for M steps. Mixing coeff is K the first 5 SCF steps.
                                           # Then it incrementally goes to L and becomes L at M-5 SCF step. From M-5 to M it's equal to L. After M'th SCF step, adaptive mixing begins.
                                           # 
                                           # [2], adaptive mixing, then pulay
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
scf pulay step   :  34 | MAX ΔE[   1]:    0.0000000 | MAX ΔDM[   0]:    0.0000008 | MAX ΔDM_ij[   1]:  0.0000072  | N not converged: 0
scf pulay step   :  24 | MAX ΔE[   1]:    0.0000003 | MAX ΔDM[   1]:    0.0000016 | MAX ΔDM_ij[   1]:  0.0000079  | N not converged: 0
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   629.51   3.254838e-01 -1.348336e+00 -1.022852e+00 ||    643.39   3.326564e-01 1.420072e+00 1.752729e+00 || 
scf pulay step   :  24 | MAX ΔE[   1]:    0.0000004 | MAX ΔDM[   1]:    0.0000020 | MAX ΔDM_ij[   1]:  0.0000090  | N not converged: 0
     2   571.53   2.955045e-01 -1.317965e+00 -1.022461e+00 ||    659.57   3.410232e-01 1.411342e+00 1.752365e+00 || 
scf pulay step   :  24 | MAX ΔE[   1]:    0.0000000 | MAX ΔDM[   0]:    0.0000012 | MAX ΔDM_ij[   1]:  0.0000086  | N not converged: 0
     3   492.78   2.547874e-01 -1.276813e+00 -1.022026e+00 ||    659.37   3.409229e-01 1.411111e+00 1.752034e+00 || 
scf pulay step   :  24

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   645.15   3.335679e-01 -2.042483e-01 1.293196e-01 ||    639.70   3.307530e-01 2.187064e+00 2.517817e+00 || 
     2   905.22   4.680368e-01 -3.387172e-01 1.293196e-01 ||    637.33   3.295262e-01 2.188291e+00 2.517817e+00 || 
     3  1201.75   6.213508e-01 -4.920312e-01 1.293196e-01 ||    714.04   3.691856e-01 2.148632e+00 2.517817e+00 || 
     4  1523.31   7.876113e-01 -6.582917e-01 1.293196e-01 ||    866.69   4.481109e-01 2.069707e+00 2.517817e+00 || 
     5  1853.90   9.585416e-01 -8.292220e-01 1.293196e-01 ||   1085.14   5.610601e-01 1.956757e+00 2.517817e+00 || 
     6  2172.47   1.123254e+00 -9.939347e-01 1.293196e-01 ||   1351.28   6.986636e-01 1.819154e+00 2.517817e+00 || 
     7  2453.28   1.268445e+00 -1.139126e+00 1.293196e-01 ||   1637.99   8.469085e-01 1.670909e+00 2.517817e+00 || 
     8  2667.46   1.379182e+00 -1.249862e+00 1.293196e-01 ||   1908.60   9.868246e-01 1.530993e+00 2.

## Scale velocities to control temperature

In [6]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -3.774504e-02 1.690710e-01 ||    400.00   2.068160e-01 2.201857e+00 2.408673e+00 || 
     2   400.00   2.068160e-01 3.571968e-02 2.425357e-01 ||    400.00   2.068160e-01 2.218095e+00 2.424911e+00 || 
     3   400.00   2.068160e-01 1.029274e-01 3.097434e-01 ||    400.00   2.068160e-01 2.196390e+00 2.403206e+00 || 
     4   400.00   2.068160e-01 1.628647e-01 3.696807e-01 ||    400.00   2.068160e-01 2.143044e+00 2.349860e+00 || 
     5   400.00   2.068160e-01 2.110934e-01 4.179094e-01 ||    400.00   2.068160e-01 2.067636e+00 2.274452e+00 || 
     6   400.00   2.068160e-01 2.362038e-01 4.430198e-01 ||    400.00   2.068160e-01 1.978782e+00 2.185598e+00 || 
     7   400.00   2.068160e-01 2.198870e-01 4.267030e-01 ||    400.00   2.068160e-01 1.882775e+00 2.089591e+00 || 
     8   400.00   2.068160e-01 1.478116e-01 3.546276e-01 ||    400.00   2.068160e-01 1.783950e+00 1.990766e

## Langevin Thermostat

In [7]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   241.66   1.249498e-01 -1.359684e+00 -1.234734e+00 ||    971.12   5.021072e-01 1.362126e+00 1.864233e+00 || 
     2   210.49   1.088307e-01 -1.351617e+00 -1.242786e+00 ||   1077.78   5.572535e-01 1.297665e+00 1.854918e+00 || 
     3   172.02   8.894289e-02 -1.339831e+00 -1.250888e+00 ||   1123.39   5.808386e-01 1.250922e+00 1.831761e+00 || 
     4   134.46   6.952362e-02 -1.326026e+00 -1.256502e+00 ||   1145.99   5.925243e-01 1.225696e+00 1.818221e+00 || 
     5   101.91   5.269244e-02 -1.314902e+00 -1.262210e+00 ||   1114.22   5.760939e-01 1.231117e+00 1.807211e+00 || 
     6    81.99   4.239109e-02 -1.304010e+00 -1.261619e+00 ||   1018.96   5.268427e-01 1.268060e+00 1.794903e+00 || 
     7    71.38   3.690787e-02 -1.296128e+00 -1.259220e+00 ||    869.80   4.497204e-01 1.332704e+00 1.782425e+00 || 
     8    64.34   3.326482e-02 -1.292263e+00 -1.258999e+00 ||    675.94   3.494870e-01 1.41640